In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import time
import requests
import random
import aiohttp

In [2]:
async def getJSONData(full_url):
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(full_url) as r:
                if r.status == 200:
                    js = await r.json()
                    return js
    except:
        return None


In [3]:
def getTrackData(player_id):
    url = 'https://mkwlounge.gg/ladder/player.php?player_id={}&ladder_id=3'.format(player_id)
    # print(url)
    playerpage = requests.get(url, timeout=3)
    playersoup = bs(playerpage.content)
    playertables = playersoup.find_all('table')
    dfs = pd.read_html(str(playertables))[3]
    return dfs

In [4]:
def formatTrackData(df, index):
    df.drop(['Std Dev Time', 'Average Points', 'Fastest Time', 'Slowest Time'], axis=1, inplace=True)
    melted = df.pivot(index = 'Engine', columns='Track')
    melted = melted.drop('Mirror', axis=0, errors='ignore')
    melted = melted.drop('100cc', axis=0, errors='ignore')
    melted = melted.rename(index={'150cc': index})
    return melted

In [5]:
upperDict = {'Average Placement':'AP',
             'Average Time': 'AT',
             'Races': 'R',
             'player_id':'player_id'}
def renameCols(df):
    df.rename(columns=lambda x: upperDict[x], level=0, inplace=True)
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    df.drop(['AT -','AP -', 'R -'], axis=1, inplace=True)
    df.rename(columns=lambda c: c.replace(' (Nintendo)',''), inplace=True)
    df.index.name = None
    return df

In [16]:
async def getPlayerData():
    metadata = await getJSONData('https://www.mkwlounge.gg/api/ladderplayer.php?ladder_type=rt&all=1')
    playerdata = pd.DataFrame(metadata['results'])
    playerdata = playerdata[['player_id','player_name','current_mmr','current_lr','win_percentage','win10_percentage','average_score','average10_score','total_events']]
    playerdata = playerdata.loc[playerdata["total_events"]!= 0]
    playerdata = playerdata.loc[playerdata["current_mmr"] >= 0]
    return playerdata

In [17]:
playerdata = await getPlayerData()
playerdata.head()

C:\Users\rford\AppData\Local\Temp/ipykernel_20804/3094203558.py:1: RuntimeWarning: coroutine 'getPlayerData' was never awaited
  playerdata = await getPlayerData()


,player_id,player_name,current_mmr,current_lr,win_percentage,win10_percentage,average_score,average10_score,total_events
0,260,Az,17155,15757,0.962963,1.000000,99.0370,96.1000,54
1,1207,Kevin,14339,15268,0.769841,0.555556,89.2217,78.7500,127
2,290,barney,14495,14979,0.784946,0.600000,91.0257,90.7000,94
3,1614,Kasperinos,13902,14817,0.758242,0.400000,90.2771,76.9286,92
4,1043,Fox,13837,14535,0.684783,0.200000,92.1954,83.9000,92


In [18]:


dfs = []
i =0
for index, row in playerdata.iterrows():
    trackdata = getTrackData(row['player_id'])
    formatData = formatTrackData(trackdata, index)
    formatData['player_id'] = row['player_id']
    dfs.append(formatData)
    # if i %100 == 0:
    #     print('working: ' + str(i))
    # i = i + 1

trackdata = pd.DataFrame().append(dfs)
trackdata = renameCols(trackdata)
trackdata.head()

,AP DS Delfino Square,AP DS Desert Hills,AP DS Peach Gardens,AP DS Yoshi Falls,AP GBA Bowser Castle 3,AP GBA Shy Guy Beach,AP GCN DK Mountain,AP GCN Mario Circuit,AP GCN Peach Beach,AP GCN Waluigi Stadium,...,R Wii Luigi Circuit,R Wii Maple Treeway,R Wii Mario Circuit,R Wii Moo Moo Meadows,R Wii Moonview Highway,R Wii Mushroom Gorge,R Wii Rainbow Road,R Wii Toad's Factory,R Wii Wario's Gold Mine,player_id
0,5th,5th,5th,4th,5th,3rd,5th,4th,5th,5th,...,46.0,59.0,24.0,17.0,97.0,128.0,16.0,39.0,14.0,260
1,6th,5th,5th,5th,5th,5th,5th,6th,6th,6th,...,146.0,121.0,61.0,24.0,217.0,304.0,36.0,79.0,54.0,1207
2,6th,5th,6th,4th,5th,9th,5th,5th,6th,5th,...,61.0,67.0,22.0,21.0,87.0,122.0,18.0,42.0,34.0,290
3,5th,5th,5th,5th,5th,5th,5th,7th,5th,5th,...,55.0,43.0,26.0,19.0,102.0,104.0,7.0,36.0,22.0,1614
4,6th,5th,5th,6th,5th,3rd,4th,6th,6th,5th,...,58.0,54.0,20.0,15.0,62.0,104.0,13.0,31.0,22.0,1043


In [20]:
combodata = pd.merge(playerdata,trackdata, left_on='player_id', right_on='player_id')
combodata.head()

,player_id,player_name,current_mmr,current_lr,win_percentage,win10_percentage,average_score,average10_score,total_events,AP DS Delfino Square,...,R Wii Koopa Cape,R Wii Luigi Circuit,R Wii Maple Treeway,R Wii Mario Circuit,R Wii Moo Moo Meadows,R Wii Moonview Highway,R Wii Mushroom Gorge,R Wii Rainbow Road,R Wii Toad's Factory,R Wii Wario's Gold Mine
0,260,Az,17155,15757,0.962963,1.000000,99.0370,96.1000,54,5th,...,61.0,46.0,59.0,24.0,17.0,97.0,128.0,16.0,39.0,14.0
1,1207,Kevin,14339,15268,0.769841,0.555556,89.2217,78.7500,127,6th,...,161.0,146.0,121.0,61.0,24.0,217.0,304.0,36.0,79.0,54.0
2,290,barney,14495,14979,0.784946,0.600000,91.0257,90.7000,94,6th,...,94.0,61.0,67.0,22.0,21.0,87.0,122.0,18.0,42.0,34.0
3,1614,Kasperinos,13902,14817,0.758242,0.400000,90.2771,76.9286,92,5th,...,73.0,55.0,43.0,26.0,19.0,102.0,104.0,7.0,36.0,22.0
4,1043,Fox,13837,14535,0.684783,0.200000,92.1954,83.9000,92,6th,...,70.0,58.0,54.0,20.0,15.0,62.0,104.0,13.0,31.0,22.0
